# U.S. Medical Insurance Costs

## Introduction

The data is in the file `insurance.csv`.

Objectives:
- What is correlated to high medical costs?
- What is correlated to low medical costs?
- Are there weird correlations, like smoking and children? Do parents of children smoke more?

Variables are:
- age, sex, bmi, children, smoker, region, **charges**

Basics:
- Ranges of each variable: min / max and average.
- Percentages of smokers, regions, children?


Correlations: Pairs of variables to look at:
- Charge average by : age (brackets), bmi (brackets?), children, smoker, region
    Basic conclusion : factors that increase or decrease price

- silly ones : children/smoker, bmi/smoker, age/smoker


Triple variable:?

\\ Questions?

?? Brackets for age and charges.
Maybe create a male and female data set.
Do I need to give an id to each row?
I think it might be necessary to avoid using the same row twice  when pulling from several tables.


**Notes before getting started :**
- Don't forget to import libraries csv and ???

## Roadmap:

### Setup & Prepare data
import csv library
import csv file and turn it into a dictionary of everything

#### Indexed dictionaries:
- A full indexed dictionary
- a male and female, sub-dictionary (indexed)


#### Create new csv files
- An indexed csv
- A male indexed csv and a female indexed csv


### Explore Data:
Let's start counting things:
- number of males, females, smokers, by regions, etc...
-




### Setup & Prepare Data:
I now have 3 dictionaries:
- indexed_insurance_data
- male_insurance_data
- female_insurance-data

And 3 new csv files:
- indexed_insurance.csv
- male_indexed_insurance.csv
- female_indexed_insurance.csv

In [2]:
import csv
from itertools import count

### I want to import my file insurance.csv let's see if i can see it.
insurance_data = csv.DictReader(open("/insurance.csv"))
# print(insurance_data)                     ## check/debug line

## I want to have a giant dictionary of all the data
indexed_insurance_data = {}
id = 1
for row in insurance_data:
    indexed_insurance_data[id] = row
    id += 1
#print(indexed_insurance_data)             ## check/debug line


## I want to create a dictionary of all the males.
males_insurance_data = {}
for key, record in indexed_insurance_data.items():
    # print(key, record)                   ## debug line
    if record.get('sex') == 'male':
        males_insurance_data[key] = record
# print(males_insurance_data)

## A Dictionary of females:
females_insurance_data = {}
for key, record in indexed_insurance_data.items():
    # print(key, record) ## debug line
    if record.get('sex') == 'female':
        females_insurance_data[key] = record
# print(females_insurance_data)         ## check/debug line

### Let's write new csv files : the indexed version, male and female version
## scoping this wide, we'll reuse it.
fields = ['id', 'age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']


 ## Tricky : we first make the rows using all the fields (adding 'id'). Then, we can just write the record, but then we're ## missing the id. The column exists, but the field is empty. So we do this hack: create a row = id: index dictionary. to which we update the rest of the dictionary. We can write the whole row as a dictionary.

 # create a new csv file with a new column for the id.
with open("/indexed_insurance.csv", "w") as output_csv:
    output_writer = csv.DictWriter(output_csv, fieldnames=fields)
    output_writer.writeheader()
    for index, record in indexed_insurance_data.items():
        row = {'id': index}
        row.update(record)
        output_writer.writerow(row)

file_path = "/male_indexed_insurance.csv"
with open(file_path, 'w')as f:
    output_writer = csv.DictWriter(f, fieldnames=fields)
    output_writer.writeheader()
    for index, record in males_insurance_data.items():
        row={'id': index}
        row.update(record)
        output_writer.writerow(row)


file_path = "/female_indexed_insurance.csv"
with open(file_path, 'w')as f:
    output_writer = csv.DictWriter(f, fieldnames=fields)
    output_writer.writeheader()
    for index, record in females_insurance_data.items():
        row={'id': index}
        row.update(record)
        output_writer.writerow(row)



from collections import Counter


### Explore Data
We have cleanish data.

let's explore. i want to know the ranges of things. number of smokers,




In [10]:
# let's count the males and females.
number_of_males = 0
for i in males_insurance_data.keys():
    number_of_males += 1
print(number_of_males)  ## returns 676


number_of_females = 0
for i in females_insurance_data.keys():
    number_of_females += 1
print(number_of_females) ## returns 662

## let's check using the main dictionary
num_fem = 0
num_mal = 0
for record in indexed_insurance_data.values():
        if record.get('sex') == 'female':
            num_fem += 1
        elif record.get('sex') == 'male':
            num_mal += 1
print(f"Females = {num_fem}, Males = {num_mal}")


### Let's count ocurences of some variables

sex_counter = Counter(record['sex'] for record in indexed_insurance_data.values())
print(sex_counter)

region_counter = Counter(record['region'] for record in indexed_insurance_data.values())
print(region_counter)

smoker_counter = Counter(record['smoker'] for record in indexed_insurance_data.values())
print(f"Is this person a smoker? No: {smoker_counter['no']}, Yes: {smoker_counter['yes']}")

children_counter = Counter(record['children'] for record in indexed_insurance_data.values())
print(children_counter)




676
662
Females = 662, Males = 676
Counter({'male': 676, 'female': 662})
Counter({'southeast': 364, 'southwest': 325, 'northwest': 325, 'northeast': 324})
Is this person a smoker? No: 1064, Yes: 274
Counter({'0': 574, '1': 324, '2': 240, '3': 157, '4': 25, '5': 18})


### Pre-analyse:
I have the following dictionaries for single variables:
- indexed_charges {id : charges}
- indexed_bmi  {id : bmi}
- indexed_age  {id : age}

In [39]:
## So I want to have a dictionary of id + charges

## this gives a dict of id + charges
indexed_charges = {id: data['charges'] for id, data in indexed_insurance_data.items()}
# print(indexed_charges)

## This is another way to do it that i like less.
# ind_char = {}
# for index, record in indexed_insurance_data.items():
#     ind_char[index] = record['charges']
# #print(ind_char)

## This prints the first ten values of the dict. I would like something that simple, but with the key attached
# range_ten = range(1,11)
# for i in range_ten:
#     print(i, indexed_charges[i])

## this gives a dict of id + bmi
indexed_bmi = {id : data['bmi'] for id, data in indexed_insurance_data.items()}
#print(indexed_bmi)

## this gives a dict of id + age
indexed_age = {id : data['age'] for id, data in indexed_insurance_data.items()}
print(indexed_age)


### Now, let's use dict reader to generate it for regions.




{1: '19', 2: '18', 3: '28', 4: '33', 5: '32', 6: '31', 7: '46', 8: '37', 9: '37', 10: '60', 11: '25', 12: '62', 13: '23', 14: '56', 15: '27', 16: '19', 17: '52', 18: '23', 19: '56', 20: '30', 21: '60', 22: '30', 23: '18', 24: '34', 25: '37', 26: '59', 27: '63', 28: '55', 29: '23', 30: '31', 31: '22', 32: '18', 33: '19', 34: '63', 35: '28', 36: '19', 37: '62', 38: '26', 39: '35', 40: '60', 41: '24', 42: '31', 43: '41', 44: '37', 45: '38', 46: '55', 47: '18', 48: '28', 49: '60', 50: '36', 51: '18', 52: '21', 53: '48', 54: '36', 55: '40', 56: '58', 57: '58', 58: '18', 59: '53', 60: '34', 61: '43', 62: '25', 63: '64', 64: '28', 65: '20', 66: '19', 67: '61', 68: '40', 69: '40', 70: '28', 71: '27', 72: '31', 73: '53', 74: '58', 75: '44', 76: '57', 77: '29', 78: '21', 79: '22', 80: '41', 81: '31', 82: '45', 83: '22', 84: '48', 85: '37', 86: '45', 87: '57', 88: '56', 89: '46', 90: '55', 91: '21', 92: '53', 93: '59', 94: '35', 95: '64', 96: '28', 97: '54', 98: '55', 99: '56', 100: '38', 101: '4